In [ ]:
import matplotlib as mpl
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import shapely
from scipy.spatial import cKDTree
from shapely.geometry import LineString, Polygon, MultiPolygon, Point
import alphashape
import sys
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

mpl.rcParams.update({
    'font.sans-serif': 'Arial',
    'savefig.dpi': 600,
    'figure.dpi': 150,
    'font.size': 7,
    'figure.figsize': [4, 2.5],
    'axes.labelpad': 5,
    'axes.linewidth': 0.5,
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'axes.titlesize': 'medium',
    'legend.fontsize': 'medium',
    'text.usetex': False,
    'mathtext.fontset': 'stixsans',
    'mathtext.default': 'it',
})

pd.options.display.max_columns = 50

# import warnings
# warnings.filterwarnings('ignore')

centimeters = 1 / 2.54

In [ ]:
from src.load_data import load_transects

centerline = pd.read_csv('1_input-data/centerline-points-with-idx.csv')
all_transects = load_transects('1_input-data/join_*-transect-25m.csv')
all_transects

ones that look a bit weird

22
51
59

#### clean and prep

In [ ]:
all_transects['survey_date'] = pd.to_datetime(all_transects['survey_date'], format='%Y-%m-%d', errors='coerce')
all_transects = all_transects.sort_values(['survey_date', 'transect_idx', 'z']).reset_index(drop=True)

centerline = centerline.rename(columns={'x': 'centerline_x', 'y': 'centerline_y'})
all_transects = all_transects.merge(centerline, on=['transect_idx'], how='left')

# calculate projected distance along transect

all_transects['linear_distance'] = np.sqrt(
    (all_transects['x'] - all_transects['centerline_x'])**2 +
    (all_transects['y'] - all_transects['centerline_y'])**2
)

all_transects['d_along_transect'] = np.sqrt(all_transects['linear_distance']**2 - all_transects['d_transect']**2)

d_along_transect_min = (
    all_transects
    .groupby('transect_idx')['d_along_transect']
    .transform('min')
)

all_transects['d_along_transect_adj'] = all_transects['d_along_transect'] - d_along_transect_min

In [ ]:
# remove transects

all_transects = all_transects[~all_transects['transect_idx'].isin([1, 2])].copy() # beginning bend 1 before bank edge mapping starts
all_transects = all_transects[~all_transects['transect_idx'].isin([25, 33])].copy() # bend 1 floodplain channels
all_transects = all_transects[~all_transects['transect_idx'].isin([38, 39])].copy() # bend 1 culvert and pile of tires
all_transects = all_transects[~all_transects['transect_idx'].isin([64, 65, 66])].copy() # bend 1 sunken boat
all_transects = all_transects[~all_transects['transect_idx'].isin([79, 80, 81, 82])].copy() # last transects bend 1 actually on point bar

# set max distance from transect to use points from

max_transect_width = 1
all_transects = all_transects[all_transects['d_transect'] <= max_transect_width / 2].copy()

# remove points over max elevation that likely are misclassified trees

all_transects = all_transects[all_transects['z'] < 11.5].copy()

In [ ]:
all_transects[all_transects['bend'] == 2]['transect_idx'].unique().shape

In [ ]:
all_transects[(all_transects['z'] > 11.5)  ]

### helper functions

In [ ]:
def subset_df(df, transect_idx, date):
    date = pd.to_datetime(date)
    return df[(df['transect_idx'] == transect_idx) & (df['survey_date'] == date)]

STYLE_ORIG = dict(marker='.', linestyle='none', alpha=0.2, markersize=2)
STYLE_FILT = dict(marker='.', linestyle='none', alpha=1, markersize=1)

### apply box median filter

#### apply for chosen dates and transects

In [ ]:
def box_median_coords(df, x, z, dx, dz):
    coords = df[[x, z]].to_numpy()
    tree = cKDTree(coords)
    r = max(dx, dz)
    x_meds, z_meds = [], []

    for xi, zi in coords:
        x0, x1 = xi - dx / 2, xi + dx / 2
        z0, z1 = zi - dz / 2, zi + dz / 2
        idx = [j for j in tree.query_ball_point([xi, zi], r)
               if x0 <= coords[j, 0] <= x1 and z0 <= coords[j, 1] <= z1]

        if idx:
            x_meds.append(np.median(df.iloc[idx][x]))
            z_meds.append(np.median(df.iloc[idx][z]))
        else:
            x_meds.append(xi)
            z_meds.append(zi)

    f = df.assign(x_box_median=x_meds, z_box_median=z_meds)

    # # re-add original coords of min z and max x points to try to keep existing range
    # min_z_idx = f[f[z] == f[z].min()].index[0]
    # max_x_idx = f[f[x] == f[x].max()].index[0]
    # f.loc[min_z_idx, ['x_box_median', 'z_box_median']] = f.loc[min_z_idx, [x, z]].values
    # f.loc[max_x_idx, ['x_box_median', 'z_box_median']] = f.loc[max_x_idx, [x, z]].values

    # # re-add original coords of point with max x and max z only if it is the maximum overall
    # max_both_idx = f[(f[x] == f[x].max()) & (f[z] == f[z].max())].index
    # max_both_idx = max_both_idx[0] if len(max_both_idx) > 0 else None

    if (f[x].max() > f['x_box_median'].max()) and (f[z].max() > f['z_box_median'].max()):
        max_both_idx = f[(f[x] == f[x].max()) & (f[z] == f[z].max())].index
        max_both_idx = max_both_idx[0] if len(max_both_idx) > 0 else None
    else:
        max_both_idx = None

    if max_both_idx is not None:
        f.loc[max_both_idx, ['x_box_median', 'z_box_median']] = f.loc[max_both_idx, [x, z]].values
        
    return f

In [ ]:
# choose a transect index and dates to plot with variable box size

def box_filter_and_plot_transect(df, x, z, transect_idx, dates, box_sizes=[(0.5, 0.5), (1, 1)]):
    fig, ax = plt.subplots(dpi=150)

    for date in dates:
        subset = subset_df(df, transect_idx, date)
        date = pd.to_datetime(date)

        ax.plot(subset[x], subset[z], label=f'{date.date()}', **{**STYLE_ORIG, 'markersize': 1})

        for dx, dz in box_sizes:
            f = box_median_coords(subset, x=x, z=z, dx=dx, dz=dz)
            f = f.dropna(subset=['x_box_median', 'z_box_median'])

            label = f'{date.date()} filtered ({dx}×{dz} m)'
            ax.plot(f['x_box_median'], f['z_box_median'], label=label, **{**STYLE_FILT, 'markersize': 2, 'mew':0})

    ax.set(xlabel='projected distance along centerline (m)', ylabel='z (m)',
           title=f'transect {transect_idx} box median filter')
    ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0)
    plt.show()

In [ ]:
transect_ids = np.sort(all_transects['transect_idx'].unique())[3::100]
dates = ['2022-04-15', '2024-09-06']

for tid in transect_ids:
    box_filter_and_plot_transect(all_transects, 'd_along_transect_adj', 'z', transect_idx=tid, dates=dates, box_sizes=[ (0.5, 3), (1, 3)])

In [ ]:
# double median filter could be nice but not working at the moment

# transect_ids = np.sort(all_transects['transect_idx'].unique())[3::100]
# dates = ['2022-04-15', '2024-09-06']
# median_box_1 = [(0.5, 3), (1, 3)]
# median_box_2 = [(0.5, 1), (1, 1)]

# all_filtered = {}

# for tid in transect_ids:
#     all_filtered[tid] = box_filter_and_plot_transect(all_transects, 'd_along_transect_adj', 'z', transect_idx=tid, dates=dates, box_sizes=median_box_1)

# for date in dates:
#     for box_1_size in median_box_1:
#         filtered_df = all_filtered[tid][pd.to_datetime(date)][box_1_size]
#         for box_2_size in median_box_2:
#             twice_filtered = box_median_coords(filtered_df, x='x_box_median', z='z_box_median', dx=box_2_size[0], dz=box_2_size[1])

#### apply for all dates and transects and export

would be great to make this check the folder first and only apply filter to those that are missing

In [ ]:
# box_filtered_dfs = []
# dx, dz = 1, 3  # meters

# for date in pd.to_datetime(all_transects['survey_date'].unique()):
#     transects = all_transects[all_transects['survey_date'] == date]['transect_idx'].unique()
#     n = len(transects)

#     print(f'📅 {date.date()} | 0/{n} starting...', end='\r')

#     for i, t_id in enumerate(transects, 1):
#         print(f'📅 {date.date()} transect {i}/{n} filtered'.ljust(60), end='\r')

#         subset = all_transects[(all_transects['survey_date'] == date) &(all_transects['transect_idx'] == t_id)]
#         f = box_median_coords(subset, x='d_along_transect_adj', z='z', dx=dx, dz=dz)
#         f['filter_dx'], f['filter_dz'] = dx, dz
#         box_filtered_dfs.append(f)

#     print(f'✅ {date.date()} | {n}/{n} done'.ljust(60))

# # export

# df_filtered_all = pd.concat(box_filtered_dfs, ignore_index=True)
# df_filtered_all.to_parquet('data/transects_box_filtered.parquet', index=False) 

#### import median filtered data

In [ ]:
all_transects_filt = pd.read_parquet('data/transects_box_filtered.parquet')
all_transects_filt['bend'] = (all_transects_filt['transect_idx'] > 80).astype(int) + 1
all_transects_filt

#### plot chosen dates from all data 

In [ ]:
def plot_box_filtered_transects(df, x, z, transect_idx, dates):
    fig, ax = plt.subplots()

    for date in dates:
        date = pd.to_datetime(date)
        subset = subset_df(df, transect_idx, date)

        ax.plot(subset[x], subset[z], label=f'{date.date()}', **STYLE_ORIG)

        f = subset.dropna(subset=[x, z])
        dx_vals = f['filter_dx'].unique()
        dz_vals = f['filter_dz'].unique()

        dx, dz = dx_vals[0], dz_vals[0]
        label = f'{date.date()} filtered ({dx}×{dz} m)'
        ax.plot(f[x], f[z], label=label, **STYLE_FILT)

    ax.set(xlabel='projected distance along centerline (m)', ylabel='z (m)',
           title=f'transect {transect_idx} box median filter')
    ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0)
    ax.set_xlim(-1, 25)
    plt.show()

In [ ]:
sorted_transect_idxs = np.sort(all_transects_filt['transect_idx'].unique())[94::100]
sorted_transect_idxs = [105]
dates = ['2022-09-21', '2023-01-07']
dates = ['2022-04-15', '2022-06-17', '2022-09-21', '2023-01-07', '2023-12-08']
dates = ['2024-02-27', '2024-09-06', '2025-04-04', '2025-08-22']

for idx in sorted_transect_idxs:
    plot_box_filtered_transects(all_transects_filt, 'x_box_median', 'z_box_median', transect_idx=idx, dates=dates)

### add zmax points per transect

In [ ]:
# old version, adds points to overall zmax regardless of survey timing

# def add_zmax_points_per_transect(df, x, z, transect_idx):
#     transect_data = df[df['transect_idx'] == transect_idx]
#     overall_zmax = transect_data[z].max()

#     new_points = []
#     for date in transect_data['survey_date'].dropna().unique():
#         date_data = transect_data[transect_data['survey_date'] == pd.to_datetime(date)]

#         if date_data[z].max() < overall_zmax:
#             xmax_idx = date_data[x].idxmax()

#             # add offset to z for piecewise sorting
#             new_point = {
#                 x: date_data.loc[xmax_idx, x],
#                 z: overall_zmax + 0.01,
#                 'transect_idx': transect_idx,
#                 'survey_date': date,
#                 'added_zmax': True,
#                 'zmax_height': overall_zmax - date_data[z].max()
#             }
#             new_points.append(new_point)
            
#     return pd.DataFrame(new_points)

In [ ]:
def add_zmax_points_per_transect(df, x, z, transect_idx):
    transect_data = df[df['transect_idx'] == transect_idx]
    
    # sort dates chronologically to establish survey order
    dates = sorted(transect_data['survey_date'].dropna().unique())
    
    new_points = []
    
    for i, date in enumerate(dates):
        date_data = transect_data[transect_data['survey_date'] == pd.to_datetime(date)]
        
        # first survey: use original logic (no previous surveys exist)
        if i == 0:
            overall_zmax = transect_data[z].max()
            if date_data[z].max() < overall_zmax:
                xmax_idx = date_data[x].idxmax()
                new_point = {
                    x: date_data.loc[xmax_idx, x],
                    z: overall_zmax + 0.01,
                    'transect_idx': transect_idx,
                    'survey_date': date,
                    'added_upper_z': 'overall',
                    'added_z_height': overall_zmax - date_data[z].max(),
                    'added_from_date': None
                }
                new_points.append(new_point)
        else:
            # for subsequent surveys: check if previous surveys have 
            # points extending further and higher than current z.max() point
            zmax_idx = date_data[z].idxmax()
            current_zmax = date_data.loc[zmax_idx, z]
            current_x_at_zmax = date_data.loc[zmax_idx, x]
            
            # search previous surveys (most recent first) for suitable points
            suitable_points = None
            
            for prev_date in reversed(dates[:i]):
                prev_date_data = transect_data[transect_data['survey_date'] == pd.to_datetime(prev_date)]
                
                # find points in previous survey with both higher x and higher z
                # than the current survey's highest elevation point
                candidate_points = prev_date_data[
                    (prev_date_data[x] > current_x_at_zmax) & 
                    (prev_date_data[z] > current_zmax)
                ]
                
                if not candidate_points.empty:
                    suitable_points = candidate_points
                    break  # use most recent survey that satisfies conditions
            
            # add qualifying points from previous survey, labeled with current date
            if suitable_points is not None:
                for _, row in suitable_points.iterrows():
                    new_point = {
                        x: row[x],
                        z: row[z],
                        'transect_idx': transect_idx,
                        'survey_date': date,  # label with current survey date
                        'added_upper_z': 'previous',
                        'added_z_height': row[z] - current_zmax,
                        'added_from_date': row['survey_date']
                    }
                    new_points.append(new_point)
            else:
                # fallback: use original logic if no suitable previous points found
                overall_zmax = transect_data[z].max()
                if date_data[z].max() < overall_zmax:
                    xmax_idx = date_data[x].idxmax()
                    new_point = {
                        x: date_data.loc[xmax_idx, x],
                        z: overall_zmax + 0.01,
                        'transect_idx': transect_idx,
                        'survey_date': date,
                        'added_upper_z': 'overall',
                        'added_z_height': overall_zmax - date_data[z].max(),
                        'added_from_date': None
                    }
                    new_points.append(new_point)
    
    return pd.DataFrame(new_points)

In [ ]:
all_new_points = []
df = all_transects_filt

for transect in df.transect_idx.unique():
    new_points = add_zmax_points_per_transect(df, 'x_box_median', 'z_box_median', transect)
    all_new_points.append(new_points)

all_transects_filt_add_max = pd.concat([df] + all_new_points, ignore_index=True)

In [ ]:
all_transects_filt_add_max.added_z_height.describe()

In [ ]:
fig, ax = plt.subplots(figsize=(9, 2.5))
sns.boxplot(all_transects_filt_add_max, x='transect_idx', y='added_z_height', ax=ax)

### piecewise interpolation in z

In [ ]:
def piecewise_sample_transect(df, x, z, transect_idx, date, dz=0.05):
    """
    linearly interpolate x as a function of z for a single transect and date.
    returns sampled points at even spacing in z, clipped to observed z range.
    """
    date = pd.to_datetime(date)
    subset = df[(df['transect_idx'] == transect_idx) & (df['survey_date'] == date)].dropna(subset=[x, z])
    subset = subset.sort_values(by=z)

    z_vals = subset[z].values
    x_vals = subset[x].values
    window_size = 0.5 # meters

    # remove outliers with big jumps in x using spatial window
    rolling_std = []
    for i, z_point in enumerate(z_vals):
        mask = np.abs(z_vals - z_point) <= window_size / 2
        n_points = np.sum(mask)
        if n_points <= 1:
            rolling_std.append(0)  # keep points with ≤2 neighbors
        else:
            rolling_std.append(np.std(x_vals[mask]))

    subset = subset.copy()
    subset['rolling_std'] = rolling_std
    
    rolling_std = np.array(rolling_std)
    mask = rolling_std <= 0.5
    subset_filtered = subset[mask].copy()

    z_vals_filtered = subset_filtered[z].values
    x_vals_filtered = subset_filtered[x].values

    z_piecewise = np.arange(z_vals_filtered.min(), z_vals_filtered.max() + dz, dz)
    x_piecewise = np.interp(z_piecewise, z_vals_filtered, x_vals_filtered)

    sampled_transect = pd.DataFrame({
        'transect_idx': transect_idx,
        'survey_date': date,
        'z_piecewise': z_piecewise,
        'x_piecewise': np.round(x_piecewise, 3),
        'dz_piecewise': dz
    })
    return sampled_transect, subset[[z, x, 'rolling_std']]

#### apply to all data and plot

In [ ]:
all_piecewise_samples = []
dz_piecewise = 0.05 # meters
df = all_transects_filt_add_max
date_list = pd.to_datetime(df['survey_date'].unique())
# date_list = [pd.to_datetime('2022-09-21')]

for date in date_list:
    transects = df[df['survey_date'] == date]['transect_idx'].unique()
    # transects = [22]

    for t_id in transects:
        sampled, rolling_std_df = piecewise_sample_transect(df, 'x_box_median', 'z_box_median', t_id, date, dz=dz_piecewise)
        if sampled is not None:
            all_piecewise_samples.append(sampled)

all_transects_sampled = pd.concat(all_piecewise_samples, ignore_index=True)

In [ ]:
rolling_std_df.plot('x_box_median', 'z_box_median', 'scatter', c='rolling_std')

In [ ]:
# all_transects_sampled = all_transects_sampled[all_transects_sampled['transect_idx']]

In [ ]:
def plot_piecewise_sampled_vs_box_filtered_profile(df_box_filtered, df_piecewise_sampled, transect_idx, dates):
    """
    plot filtered (x_box_median, z_box_median) and interpolated (x_piecewise, z_piecewise) for a transect and date
    """
    fig, ax = plt.subplots(figsize=(7 * centimeters, 4 * centimeters))
    dates = pd.to_datetime(dates)
    colors = plt.cm.twilight_shifted(np.linspace(0, 0.9, len(dates)))

    for i, date in enumerate(dates):    
        f = df_box_filtered[
            (df_box_filtered['transect_idx'] == transect_idx) &
            (df_box_filtered['survey_date'] == date)
        ].dropna(subset=['x_box_median', 'z_box_median'])

        s = df_piecewise_sampled[(df_piecewise_sampled['transect_idx'] == transect_idx) & (df_piecewise_sampled['survey_date'] == date)]
        
        base_color = np.minimum(colors[i] + 0.1, 1.0)
        dark_color = base_color * 0.7

        ax.plot(f['x_box_median'], f['z_box_median'], '.', alpha=0.15, markersize=6, color=base_color, mew=0)
        # ax.plot(s['x_piecewise'], s['z_piecewise'], '.', markersize=2, label='sampled (dz={:.2f})'.format(s['dz_piecewise'].iloc[0]))
        ax.plot(s['x_piecewise'], s['z_piecewise'], '.', markersize=2, label=date.strftime('%Y-%m-%d'), color=dark_color, mew=0, alpha=1)

    ax.set(xlabel='x (m)', ylabel='z (m)', title=f'transect {transect_idx}', 
        # xlim=(-1, 22), ylim=(4, 13)
        )
    # ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1), markerscale=3)
    ax.tick_params(axis='both', pad=3)

    os.makedirs('f2-alphashapes', exist_ok=True)
    fig.savefig(f'f2-alphashapes/transect_{transect_idx}.pdf', bbox_inches='tight', pad_inches=0.05)

    plt.show()
    # plt.close()

working but plots are updated with most recent changes and takes ~ 2 min to run

In [ ]:
dates = pd.to_datetime(all_transects_filt_add_max['survey_date'].unique())[::]
# # dates = [pd.to_datetime('2022-09-21')]
# transect_ids = all_transects_filt_add_max['transect_idx'].unique()[::]
transect_ids = [21, 60, 96, ]

for t_id in transect_ids:
    plot_piecewise_sampled_vs_box_filtered_profile(all_transects_filt_add_max, all_transects_sampled, t_id, dates)

In [ ]:
def plot_piecewise_sampled_vs_box_filtered_profile_3transects(df_box_filtered, df_piecewise_sampled, transect_idxs, dates):
    
    fig, axes = plt.subplots(1, 3, figsize=(19 * centimeters, 4 * centimeters), gridspec_kw={'wspace': 0.05})
    dates = pd.to_datetime(dates)
    colors = plt.cm.twilight_shifted(np.linspace(0, 0.9, len(dates)))
    
    all_x_vals = []
    all_z_vals = []
    
    for transect_idx in transect_idxs:
        for date in dates:
            f = df_box_filtered[
                (df_box_filtered['transect_idx'] == transect_idx) &
                (df_box_filtered['survey_date'] == date)
            ].dropna(subset=['x_box_median', 'z_box_median'])
            
            s = df_piecewise_sampled[
                (df_piecewise_sampled['transect_idx'] == transect_idx) & 
                (df_piecewise_sampled['survey_date'] == date)
            ]
            
            if not f.empty:
                all_x_vals.extend(f['x_box_median'])
                all_z_vals.extend(f['z_box_median'])
            if not s.empty:
                all_x_vals.extend(s['x_piecewise'])
                all_z_vals.extend(s['z_piecewise'])
    
    x_padding = (max(all_x_vals) - min(all_x_vals)) * 0.05
    z_padding = (max(all_z_vals) - min(all_z_vals)) * 0.05
    xlim = (min(all_x_vals) - x_padding, max(all_x_vals) + x_padding)
    zlim = (min(all_z_vals) - z_padding, max(all_z_vals) + z_padding)
    
    for plot_idx, transect_idx in enumerate(transect_idxs):
        ax = axes[plot_idx]
        
        for i, date in enumerate(dates):    
            f = df_box_filtered[
                (df_box_filtered['transect_idx'] == transect_idx) &
                (df_box_filtered['survey_date'] == date)
            ].dropna(subset=['x_box_median', 'z_box_median'])

            s = df_piecewise_sampled[
                (df_piecewise_sampled['transect_idx'] == transect_idx) & 
                (df_piecewise_sampled['survey_date'] == date)
            ]
            
            base_color = np.minimum(colors[i] + 0.1, 1.0)
            dark_color = base_color * 0.7

            if not f.empty:
                ax.plot(f['x_box_median'], f['z_box_median'], '.', alpha=0.1, markersize=6, color=base_color, mew=0, rasterized=True)
            if not s.empty:
                label = date.strftime('%Y-%m-%d') if plot_idx == 0 else None  # Only label first subplot
                ax.plot(s['x_piecewise'], s['z_piecewise'], '.', markersize=2, label=label, color=dark_color, mew=0, alpha=1, rasterized=True)

        ax.set_xlabel('x (m)')
        ax.set_ylabel('z (m)' if plot_idx == 0 else '')  # Only label y-axis for first subplot
        ax.set_title(f'transect {transect_idx}')
        ax.set_xlim(xlim)
        ax.set_ylim(zlim)
        ax.tick_params(axis='both', pad=3)
        # ax.set_xticks([1, 4, 7, 9])
        # for ax in axes:
        #     ax.grid(axis='y', linestyle='-', alpha=0.5)

        if plot_idx > 0:
            ax.set_yticks([])
            # ax.spines['left'].set_visible(False)
            # ax.spines['right'].set_visible(False)
        if plot_idx != 1:
            ax.set_xlabel('')
    
    axes[0].legend(loc='upper left', bbox_to_anchor=(3.15, 1.05), markerscale=3, handletextpad=0.2)
    plt.tight_layout()
    
    os.makedirs('f2-alphashapes', exist_ok=True)
    transect_str = '_'.join(map(str, transect_idxs))
    fig.savefig(f'f2-alphashapes/transects_{transect_str}.pdf', bbox_inches='tight', pad_inches=0.05, dpi=600)
    
    plt.show()

plot_piecewise_sampled_vs_box_filtered_profile_3transects(
    all_transects_filt_add_max, 
    all_transects_sampled, 
    [21, 60, 96], 
    dates
)

### clip data with z values below the highest river stage

In [ ]:
z_min_max = (
    all_transects_sampled
    .groupby(['transect_idx', 'survey_date'])['z_piecewise'].min()
    .groupby('transect_idx').max()
    .rename('z_min_max')
    .reset_index()
)
all_transects_sampled_clipped = all_transects_sampled.merge(z_min_max, on='transect_idx')
all_transects_sampled_clipped = all_transects_sampled_clipped[all_transects_sampled_clipped['z_piecewise'] >= all_transects_sampled_clipped['z_min_max']].copy()
all_transects_sampled_clipped = all_transects_sampled_clipped.drop(columns='z_min_max')

In [ ]:
all_transects_sampled_clipped

In [ ]:
def plot_sampled_profiles_by_date(df, transect_idx, dates):
    """
    plot x_piecewise vs z_piecewise for one transect and one or more survey dates
    """
    fig, ax = plt.subplots()

    # handle single date input
    dates = pd.to_datetime(dates) if isinstance(dates, (str, pd.Timestamp)) else pd.to_datetime(dates)

    for date in dates:
        subset = subset_df(df, transect_idx, date)
        if not subset.empty:
            ax.plot(subset['x_piecewise'], subset['z_piecewise'], label=str(date.date()), linestyle='-', marker='.')

    ax.set(xlabel='x (m)', ylabel='z (m)', title=f'transect {transect_idx}')
    ax.legend(loc='upper left')
    plt.show()

In [ ]:
all_transects_sampled_clipped

In [ ]:
plot_sampled_profiles_by_date(all_transects_sampled_clipped, transect_idx=10, dates=['2022-04-15', '2025-08-22'])

### make bounding points

In [ ]:
# # for two transects, make lines connecting the xmin points and the zmax points and sample points along those lines

# def create_bounding_points(df, transect_idx, date1, date2, x='x_piecewise', z='z_piecewise', spacing=0.1):
#     """
#     sample lines connecting xmin and zmax points between two surveys with fixed spacing
#     """
#     d1, d2 = pd.to_datetime([date1, date2])
#     f = df[(df['transect_idx'] == transect_idx) & (df['survey_date'].isin([d1, d2]))]
#     f1, f2 = f[f['survey_date'] == d1], f[f['survey_date'] == d2]

#     def get_endpoints(kind):
#         i = x if kind == 'xmin' else z
#         return f1.loc[f1[i].idxmin() if kind == 'xmin' else f1[i].idxmax()][[x, z]].values, \
#                f2.loc[f2[i].idxmin() if kind == 'xmin' else f2[i].idxmax()][[x, z]].values

#     def sample_line(p1, p2):
#         dist = np.linalg.norm(p2 - p1)
#         t = np.arange(0, 1 + spacing / dist, spacing / dist)
#         return np.column_stack([p1[0] + t * (p2[0] - p1[0]), p1[1] + t * (p2[1] - p1[1])])

#     xmin_line = sample_line(*get_endpoints('xmin'))
#     zmax_line = sample_line(*get_endpoints('zmax'))

#     out = pd.DataFrame(np.vstack([xmin_line, zmax_line]), columns=[x, z])
#     out['transect_idx'] = transect_idx
#     out['connector'] = ['xmin'] * len(xmin_line) + ['zmax'] * len(zmax_line)
#     out['survey_date_1'], out['survey_date_2'] = d1, d2

#     return out

#### calculate bounding points for one transect

In [ ]:
def create_bounding_points(df, x, z, transect_idx, date1, date2, spacing=0.1):
    d1, d2 = pd.to_datetime([date1, date2])
    f = df[(df['transect_idx'] == transect_idx) & (df['survey_date'].isin([d1, d2]))]
    f1, f2 = f[f['survey_date'] == d1], f[f['survey_date'] == d2]

    def get_endpoints(kind):
        if kind == 'xmin':
            p1 = f1.loc[f1[x].idxmin()][[x, z]].values
            p2 = f2.loc[f2[x].idxmin()][[x, z]].values
        elif kind == 'zmax':
            p1 = f1.loc[f1[z].idxmax()][[x, z]].values
            p2 = f2.loc[f2[z].idxmax()][[x, z]].values
        return p1, p2

    def sample_line(p1, p2):
        dist = np.linalg.norm(p2 - p1)
        n_points = int(dist / spacing) + 1
        t = np.linspace(0, 1, n_points)
        return np.column_stack([p1[0] + t * (p2[0] - p1[0]), p1[1] + t * (p2[1] - p1[1])])

    points = []

    xmin1 = f1[x].min()
    xmin2 = f2[x].min()
    if xmin1 < xmin2:
        xmin_line = sample_line(*get_endpoints('xmin'))
        xmin_df = pd.DataFrame(xmin_line, columns=[x, z])
        xmin_df['boundary'] = 'xmin'
        points.append(xmin_df)

    zmax_p1, zmax_p2 = get_endpoints('zmax')
    if zmax_p1[0] < zmax_p2[0]:  # compare x-coordinates
        zmax_line = sample_line(zmax_p1, zmax_p2)
        zmax_df = pd.DataFrame(zmax_line, columns=[x, z])
        zmax_df['boundary'] = 'zmax'
        points.append(zmax_df)

    if not points:
        return pd.DataFrame()

    result = pd.concat(points, ignore_index=True)
    result['transect_idx'] = transect_idx
    result['boundary_survey_date_1'] = d1
    result['boundary_survey_date_2'] = d2

    return result

In [ ]:
def plot_filtered_transect_with_connectors(df, sampled_pts, x, z, transect_idx, date1, date2):
    """
    plot box median-filtered x/z for two survey dates and the sampled bounding connector points
    """
    d1, d2 = pd.to_datetime([date1, date2])
    subset = df[(df['transect_idx'] == transect_idx) & 
                (df['survey_date'].isin([d1, d2]))]

    fig, ax = plt.subplots(dpi=200)

    for label, g in sampled_pts.groupby('boundary'):
        ax.plot(g[x], g[z], marker='.', linestyle='none', markersize=1, label=f'boundary {label}')

    for d in [d1, d2]:
        sdf = subset[subset['survey_date'] == d].dropna(subset=[x, z])
        ax.plot(sdf[x], sdf[z], label=f'{d.date()}', **STYLE_ORIG)

    ax.set(xlabel='x (m)', ylabel='z (m)', title=f'transect {transect_idx} piecewise sampled + boundaries')
    ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0)
    plt.show()

In [ ]:
all_transects_sampled_clipped

In [ ]:
boundary_spacing = dz_piecewise * 2
transect = 102
date1 = '2022-04-15'
date2 = '2025-04-04'

bounding_points = create_bounding_points(all_transects_sampled_clipped, x='x_piecewise', z='z_piecewise', transect_idx=transect, date1=date1, date2=date2, spacing=boundary_spacing)

plot_filtered_transect_with_connectors(all_transects_sampled_clipped, bounding_points, x='x_piecewise', z='z_piecewise', transect_idx=transect, date1=date1, date2=date2)

In [ ]:
print(boundary_spacing)

#### decide on which dates to use

In [ ]:
pd.to_datetime(all_transects_sampled_clipped['survey_date'].unique())

In [ ]:
# use only consecutive pairs of dates

all_dates = pd.to_datetime(all_transects_sampled_clipped['survey_date'].unique())
all_dates = np.sort(all_dates)

date_pairs_consec = {}
for transect_idx, group in all_transects_sampled_clipped.groupby('transect_idx'):
    transect_dates = sorted(group['survey_date'].unique())
    date_pairs_consec[transect_idx] = list(zip(transect_dates[:-1], transect_dates[1:]))

#### calculate for all transects

In [ ]:
def generate_bounding_points_all_transects(df, x, z, transect_ids, spacing=0.1):
    all_bounds = []

    for t_id in transect_ids:
        transect_data = df[df['transect_idx'] == t_id]
        transect_dates = sorted(transect_data['survey_date'].unique())
        
        for i in range(len(transect_dates) - 1):
            date1, date2 = transect_dates[i], transect_dates[i + 1]
            
            f1 = transect_data[transect_data['survey_date'] == date1]
            f2 = transect_data[transect_data['survey_date'] == date2]
            
            if f1.empty or f2.empty:
                continue

            bounds = create_bounding_points(df, x, z, t_id, date1, date2, spacing=spacing)
            if not bounds.empty:
                all_bounds.append(bounds)

    return pd.concat(all_bounds, ignore_index=True) if all_bounds else pd.DataFrame()

In [ ]:
transects_to_use = all_transects_sampled_clipped['transect_idx'].unique()

all_bounding_points = generate_bounding_points_all_transects(
    all_transects_sampled_clipped,
    'x_piecewise',
    'z_piecewise',
    transect_ids=transects_to_use,
    spacing=boundary_spacing
)

In [ ]:
def plot_piecewise_samples_with_bounding_points(df_piecewise, df_bounding_points, x, z, transect_idx, date1, date2):
    d1, d2 = pd.to_datetime([date1, date2])
    fig, ax = plt.subplots()

    for d in [d1, d2]:
        subset = df_piecewise[
            (df_piecewise['transect_idx'] == transect_idx) &
            (df_piecewise['survey_date'] == d)
        ]
        if not subset.empty:
            ax.plot(subset[x], subset[z], '.', label=f'{d.date()}', markersize=1)

    # plot boundary points (lines will auto-connect because of ordering)
    boundary = df_bounding_points[
        (df_bounding_points['transect_idx'] == transect_idx) &
        (df_bounding_points['boundary_survey_date_1'] == d1) &
        (df_bounding_points['boundary_survey_date_2'] == d2)
    ]

    if not boundary.empty:
        for label, g in boundary.groupby('boundary'):
            ax.plot(g[x], g[z], marker='.', label=f'boundary {label}', markersize=1, ls='none')

    ax.set(xlabel='x (m)', ylabel='z (m)',
           title=f'transect {transect_idx} {d1.date()} to {d2.date()}')
    ax.legend(loc='upper left')
    plt.show()

In [ ]:
plot_piecewise_samples_with_bounding_points(
    all_transects_sampled_clipped,
    all_bounding_points,
    'x_piecewise',
    'z_piecewise',
    transect_idx=114,
    date1='2025-04-04',
    date2='2025-08-22'
)


### aggregate boundary points and create internal points

In [ ]:
def aggregate_perimeter_points_per_transect(df_piecewise, df_bounding_points, x, z, transect_idx, date1, date2):
    d1, d2 = pd.to_datetime([date1, date2])

    transect1 = df_piecewise[(df_piecewise['transect_idx'] == transect_idx) & (df_piecewise['survey_date'] == d1)].copy()
    transect2 = df_piecewise[(df_piecewise['transect_idx'] == transect_idx) & (df_piecewise['survey_date'] == d2)].copy()

    boundary = df_bounding_points[
        (df_bounding_points['transect_idx'] == transect_idx) &
        (df_bounding_points['boundary_survey_date_1'] == d1) &
        (df_bounding_points['boundary_survey_date_2'] == d2)
    ]

    keep_cols = [x, z, 'transect_idx', 'survey_date', 'boundary_survey_date_1', 'boundary_survey_date_2', 'boundary']

    perimeter = pd.concat([transect1, transect2, boundary], ignore_index=True)[keep_cols]
    perimeter['point_type'] = 'perimeter'
    
    return perimeter

In [ ]:
def slice_and_add_random_points_to_transect(df_perimeter, x, z, slice_width=0.1, density_scale=100):
    """
    fill vertical slices in z with random points in (x, z) where x between xmin and xmax
    """
    zmin, zmax = df_perimeter[z].min(), df_perimeter[z].max()
    slices = np.arange(zmin, zmax, slice_width)
    all_random_points = []

    sorted_dates = np.sort(df_perimeter['survey_date'].dropna().unique())
    if len(sorted_dates) > 2:
        print('more than two survey dates in df!')
    earlier_survey, later_survey = sorted_dates[0], sorted_dates[1]

    for z0 in slices:
        z1 = z0 + slice_width
        slice = df_perimeter[(df_perimeter[z] >= z0) & (df_perimeter[z] < z1)]

        if slice.empty or slice[z].nunique() < 2:
            continue

        slice_earlier = slice[slice['survey_date'] == earlier_survey]
        slice_later = slice[slice['survey_date'] == later_survey]

        x_min, x_max = slice[x].min(), slice[x].max()
        x_earlier = slice_earlier[x].median()
        x_later = slice_later[x].median()

        # skip if deposition (x moved left)
        if x_later <= x_earlier or x_min == x_max:
            continue

        # assign boundary label only if slice bounds are defined by boundary-tagged points
        boundary_label = None
        boundary_points = slice[slice['boundary'].notna()]

        if not boundary_points.empty:
            bx_min, bx_max = boundary_points[x].min(), boundary_points[x].max()

            # check if bounds match those of the full slice
            x_bounds_match = np.isclose(x_min, bx_min) and np.isclose(x_max, bx_max)

            # check if those same bounds are also matched by points in slice_earlier or slice_later
            ex_min = pd.concat([slice_earlier, slice_later])[x].min()
            ex_max = pd.concat([slice_earlier, slice_later])[x].max()
            bounds_overlap_with_surveys = np.isclose(x_min, ex_min) or np.isclose(x_max, ex_max)

            if x_bounds_match and not bounds_overlap_with_surveys:
                boundary_label = boundary_points['boundary'].iloc[0]

        # generate random points within slice
        slice_area = (x_max - x_min) * (z1 - z0)
        n_points = max(1, int(slice_area * density_scale))  # linear scaling
        z_rand = np.random.uniform(z0, z1, n_points)
        x_rand = np.random.uniform(x_min, x_max, n_points)
        pts = pd.DataFrame({x: x_rand, z: z_rand,
                            'z_slice_min': np.round(z0, 2), 'z_slice_max': np.round(z1, 2)})

        if boundary_label:
            pts['boundary'] = boundary_label
        if 'transect_idx' in df_perimeter.columns:
            pts['transect_idx'] = df_perimeter['transect_idx'].iloc[0]

        all_random_points.append(pts)
        
    if all_random_points:
        random_df = pd.concat(all_random_points, ignore_index=True)
        random_df['random_survey_date_1'] = earlier_survey
        random_df['random_survey_date_2'] = later_survey
        random_df['point_type'] = 'random'
    else:
        random_df = pd.DataFrame()

    return random_df

In [ ]:
def plot_filled_with_bounds(df_perimeter, df_random):
    fig, ax = plt.subplots()

    transect_idx = df_perimeter['transect_idx'].iloc[0]
    dates = df_perimeter['survey_date'].unique().dropna()
    d1, d2 = dates[0], dates[1]

    ax.plot(df_perimeter['x_piecewise'], df_perimeter['z_piecewise'], '.', color='gray', alpha=0.6, label='boundary points')
    ax.plot(df_random['x_piecewise'], df_random['z_piecewise'], '.', color='orange', markersize=2, label='filled points')

    ax.set(title=f'transect {transect_idx}: filled area between {d1.date()} and {d2.date()}',
           xlabel='x (m)', ylabel='z (m)')
    ax.legend()
    plt.show()

In [ ]:
def remove_perimeter_point_slices_with_deposition(df_perimeter, x, z, slice_width=0.1):
    zmin, zmax = df_perimeter[z].min(), df_perimeter[z].max()
    slices = np.arange(zmin, zmax, slice_width)
    slices_to_keep = []

    sorted_dates = np.sort(df_perimeter['survey_date'].dropna().unique())

    for z0 in slices:
        z1 = z0 + slice_width
        slice_df = df_perimeter[(df_perimeter[z] >= z0) & (df_perimeter[z] < z1)]

        slice_earlier = slice_df[slice_df['survey_date'] == sorted_dates[0]]
        slice_later = slice_df[slice_df['survey_date'] == sorted_dates[1]]

        x_earlier = slice_earlier[x].median()
        x_later = slice_later[x].median()

        if x_later <= x_earlier:
            continue

        slices_to_keep.append(slice_df)

    only_erosion = pd.concat(slices_to_keep, ignore_index=True) if slices_to_keep else pd.DataFrame()

    return only_erosion

In [ ]:
transects_to_use = all_transects_sampled_clipped['transect_idx'].unique()
x = 'x_piecewise'
z = 'z_piecewise'
slice_width = 0.1  # meters

list_of_perimeter_plus_random = []

for transect_count, transect_idx in enumerate(transects_to_use, start=1):
    print(f'▶️ processing transect {transect_count}/{len(transects_to_use)} (transect_idx={transect_idx})')
    
    transect_data = all_transects_sampled_clipped[all_transects_sampled_clipped['transect_idx'] == transect_idx]
    transect_dates = sorted(transect_data['survey_date'].unique())
    
    for i in range(len(transect_dates) - 1):
        date1, date2 = transect_dates[i], transect_dates[i + 1]
        print(f'  ⏳ {date1.date()} → {date2.date()}')

        perimeter_points = aggregate_perimeter_points_per_transect(
            all_transects_sampled_clipped,
            all_bounding_points,
            x, z,
            transect_idx,
            date1, date2
        )

        cleaned_perimeter = remove_perimeter_point_slices_with_deposition(perimeter_points, x, z, slice_width)
        random_points = slice_and_add_random_points_to_transect(perimeter_points, x, z, slice_width, 150)

        perimeter_plus_random = pd.concat([cleaned_perimeter, random_points], ignore_index=True)
        list_of_perimeter_plus_random.append(perimeter_plus_random)

all_cleaned_perimeter_plus_random = pd.concat(list_of_perimeter_plus_random, ignore_index=True) if list_of_perimeter_plus_random else pd.DataFrame()
all_cleaned_perimeter_plus_random.to_parquet('data/perimeter_plus_random_points.parquet', index=False)

In [ ]:
all_cleaned_perimeter_plus_random = pd.read_parquet('data/perimeter_plus_random_points.parquet')

In [ ]:
all_bounding_points

In [ ]:
plot_transect = 113
date1 = '2024-09-06'
date2 = '2025-04-04'
x = 'x_piecewise'
z = 'z_piecewise'
slice_width = 0.1

perimeter_points = aggregate_perimeter_points_per_transect(
    all_transects_sampled_clipped,
    all_bounding_points, x=x, z=z, transect_idx=plot_transect, date1=date1, date2=date2)

random_points = slice_and_add_random_points_to_transect(perimeter_points, x, z, slice_width, 150)
cleaned_perimeter = remove_perimeter_point_slices_with_deposition(perimeter_points, x, z, slice_width)
perimeter_plus_random = pd.concat([perimeter_points, random_points], ignore_index=True)
plot_filled_with_bounds(cleaned_perimeter, random_points)

In [ ]:
filter = all_cleaned_perimeter_plus_random[
    (all_cleaned_perimeter_plus_random['transect_idx'] == plot_transect) &
    (all_cleaned_perimeter_plus_random['survey_date'].isin([pd.to_datetime(date1), pd.to_datetime(date2)]))
]
fig, ax = plt.subplots()
scatter = ax.scatter(filter[x], filter[z], c=filter['survey_date'], s=1, alpha=0.5)

unique_dates = sorted(filter['survey_date'].unique())
date_labels = [pd.to_datetime(date).strftime('%Y-%m-%d') for date in unique_dates]

handles, _ = scatter.legend_elements()
ax.legend(handles, date_labels, title="survey date")

In [ ]:
all_cleaned_perimeter_plus_random.head(2)

### alpha shape :)

In [ ]:
def get_subset(df, transect_idx, date1, date2, date_col1, date_col2=None):
    if date_col2:
        return df[(df['transect_idx'] == transect_idx) & (df[date_col1] == date1) & (df[date_col2] == date2)]
    else:
        return df[(df['transect_idx'] == transect_idx) & ((df[date_col1] == date1) | (df[date_col1] == date2))]

def deduplicate(df, x_col, z_col, tolerance=0.02):
    df['x_rounded'] = (df[x_col] / tolerance).round().astype(int)
    df['z_rounded'] = (df[z_col] / tolerance).round().astype(int)
    df = df.drop_duplicates(subset=['x_rounded', 'z_rounded'])
    df = df.drop(columns=['x_rounded', 'z_rounded'])
    return list(zip(df[x_col], df[z_col]))

In [ ]:
# old version no error handling

# alpha_val = 2.5
# alpha_shapes = []
# records = []
# df_points = all_cleaned_perimeter_plus_random

# for transect_idx, transect_data in df_points.groupby('transect_idx'):
#     all_dates = transect_data['survey_date'].dropna().unique()
#     transect_dates = sorted(all_dates)

#     for i in range(len(transect_dates) - 1):
#         date1, date2 = transect_dates[i], transect_dates[i + 1]
#         print(f' {transect_idx} ⏳ {pd.Timestamp(date1).date()} → {pd.Timestamp(date2).date()}', end='\r')

#         transects = get_subset(df_points, transect_idx, date1, date2, 'survey_date')
#         boundaries = get_subset(df_points, transect_idx, date1, date2, 'boundary_survey_date_1', 'boundary_survey_date_2')
#         random = get_subset(df_points, transect_idx, date1, date2, 'random_survey_date_1', 'random_survey_date_2')

#         aggregated = pd.concat([transects, boundaries, random], ignore_index=True)
#         points_2d = deduplicate(aggregated, 'x_piecewise', 'z_piecewise')

#         poly = alphashape.alphashape(points_2d, alpha_val)

#         area = poly.area
#         polygon_index = len(alpha_shapes)

#         alpha_shapes.append(poly)
#         records.append({
#             'transect_idx': transect_idx,
#             'alphashape_survey_date_1': date1,
#             'alphashape_survey_date_2': date2,
#             'alpha': alpha_val,
#             'area': area,
#             'polygon_index': polygon_index
#         })

# alpha_metadata = pd.DataFrame(records)

In [ ]:
# NEW CODE 7:52 PM

def compute_alphashape_with_fallback(points_2d, initial_alpha=3.0, min_alpha=1.0, min_coverage=0.7):
    """
    compute alphashape with decreasing alpha values as fallback
    returns (polygon, final_alpha, coverage_fraction, error_info)
    """
    if not isinstance(points_2d, np.ndarray):
        points_2d = np.array(points_2d)
    alpha = initial_alpha
    error_log = []
    
    # buffer distance for "close to" polygon (1% of bounding box diagonal)
    x_range = points_2d[:, 0].max() - points_2d[:, 0].min()
    z_range = points_2d[:, 1].max() - points_2d[:, 1].min()
    buffer_dist = 0.01 * np.sqrt(x_range**2 + z_range**2)
    
    while alpha >= min_alpha:
        try:
            poly = alphashape.alphashape(points_2d, alpha)
            
            # check if result is valid (not empty)
            if poly is None or poly.is_empty:
                error_log.append(f"alpha {alpha:.1f}: empty result")
                alpha -= 0.5
                continue
            
            # check point coverage
            buffered_poly = poly.buffer(buffer_dist)
            points_geom = [Point(x, z) for x, z in points_2d]
            covered_points = sum(buffered_poly.contains(pt) for pt in points_geom)
            coverage_fraction = covered_points / len(points_2d)
            
            # accept if coverage is sufficient
            if coverage_fraction >= min_coverage:
                return poly, alpha, coverage_fraction, None
            else:
                error_log.append(f"alpha {alpha:.1f}: low coverage ({coverage_fraction:.2f})")
                alpha -= 0.5
                continue
                
        except Exception as e:
            error_info = f"alpha {alpha:.1f}: {type(e).__name__}: {str(e)}"
            error_log.append(error_info)
            alpha -= 0.5
    
    # if all alpha values failed
    full_error = "; ".join(error_log)
    return None, None, None, full_error

use this for computation but parquet is updated and it is slow to run

In [ ]:
alpha_val = 3.0
alpha_shapes = []
records = []
df_points = all_cleaned_perimeter_plus_random

for transect_idx, transect_data in df_points.groupby('transect_idx'):
    all_dates = transect_data['survey_date'].dropna().unique()
    transect_dates = sorted(all_dates)

    for i in range(len(transect_dates) - 1):
        date1, date2 = transect_dates[i], transect_dates[i + 1]
        print(f' {transect_idx} ⏳ {pd.Timestamp(date1).date()} → {pd.Timestamp(date2).date()}', end='\r')

        transects = get_subset(df_points, transect_idx, date1, date2, 'survey_date')
        boundaries = get_subset(df_points, transect_idx, date1, date2, 'boundary_survey_date_1', 'boundary_survey_date_2')
        random = get_subset(df_points, transect_idx, date1, date2, 'random_survey_date_1', 'random_survey_date_2')

        aggregated = pd.concat([transects, boundaries, random], ignore_index=True)
        points_2d = deduplicate(aggregated, 'x_piecewise', 'z_piecewise')
        
        # diagnostic info
        n_points = len(points_2d)
        
        poly, final_alpha, coverage_fraction, error_info = compute_alphashape_with_fallback(points_2d, alpha_val)
        
        if poly is not None:
            area = poly.area
            polygon_index = len(alpha_shapes)
            alpha_shapes.append(poly)
            
            records.append({
                'transect_idx': transect_idx,
                'alphashape_survey_date_1': date1,
                'alphashape_survey_date_2': date2,
                'alpha': final_alpha,
                'area': area,
                'polygon_index': polygon_index,
                'n_points': n_points,
                'coverage_fraction': coverage_fraction,
                'alpha_changed': final_alpha != alpha_val,
                'error_info': None
            })
        else:
            # failed to compute alphashape
            print(f"\nFAILED: transect {transect_idx}, {pd.Timestamp(date1).date()} → {pd.Timestamp(date2).date()}")
            print(f"  points: {n_points}, errors: {error_info}")
            
            records.append({
                'transect_idx': transect_idx,
                'alphashape_survey_date_1': date1,
                'alphashape_survey_date_2': date2,
                'alpha': None,
                'area': None,
                'polygon_index': None,
                'n_points': n_points,
                'coverage_fraction': None,
                'alpha_changed': None,
                'error_info': error_info
            })

alpha_metadata = pd.DataFrame(records)

In [ ]:
alpha_metadata.to_parquet('data/alpha_metadata.parquet', index=False)

In [ ]:
alpha_metadata = pd.read_parquet('data/alpha_metadata.parquet')
alpha_metadata

In [ ]:
alpha_metadata[alpha_metadata['error_info'].notna()]

In [ ]:
alpha_metadata = alpha_metadata.dropna(subset='polygon_index')
remaining_indices = alpha_metadata['polygon_index'].astype(int).tolist().copy()

alpha_shapes = [alpha_shapes[i] for i in remaining_indices]
alpha_metadata['polygon_index'] = range(len(alpha_metadata))

In [ ]:
alpha_metadata[alpha_metadata['alpha_changed'] == True]

In [ ]:
def plot_alpha_shape_with_points(transect_idx, date1, date2, alpha_metadata, alpha_shapes, all_points):
    date1, date2 = pd.to_datetime(date1), pd.to_datetime(date2)

    match = get_subset(alpha_metadata, transect_idx, date1, date2, 'alphashape_survey_date_1', 'alphashape_survey_date_2')

    row = match.iloc[0]
    poly, area, alpha = alpha_shapes[row['polygon_index']], row['area'], row['alpha']

    transects = get_subset(all_points, transect_idx, date1, date2, 'survey_date')
    boundaries = get_subset(all_points, transect_idx, date1, date2, 'boundary_survey_date_1', 'boundary_survey_date_2')
    random = get_subset(all_points, transect_idx, date1, date2, 'random_survey_date_1', 'random_survey_date_2')

    aggregated = pd.concat([transects, boundaries, random], ignore_index=True)

    fig, ax = plt.subplots(dpi=150)

    if isinstance(poly, Polygon):
        x, y = poly.exterior.xy
        ax.fill(x, y, alpha=0.3, facecolor='gray', edgecolor='black', label='')
    elif isinstance(poly, MultiPolygon):
        for p in poly.geoms:
            x, y = p.exterior.xy
            ax.fill(x, y, alpha=0.3, facecolor='gray', edgecolor='black', label='')

    ax.plot(transects['x_piecewise'], transects['z_piecewise'], '.', markersize=1, label='transects', alpha=1)
    ax.plot(boundaries['x_piecewise'], boundaries['z_piecewise'], '.', markersize=1, label='boundaries', alpha=1)
    ax.plot(random['x_piecewise'], random['z_piecewise'], '.', markersize=1, label='random', alpha=1)

    ax.set_title(f'transect {transect_idx}, {date1.date()} → {date2.date()}\nalpha = {alpha}, area = {area:.2f} m²')
    ax.set_xlabel('x (m)')
    ax.set_ylabel('z (m)')
    ax.legend(loc='upper left')
    ax.set_aspect('equal')
    plt.show()

In [ ]:
plot_alpha_shape_with_points(
    transect_idx=61,
    date1='2022-09-21',
    date2='2023-01-07',
    alpha_metadata=alpha_metadata,
    alpha_shapes=alpha_shapes,
    all_points=all_cleaned_perimeter_plus_random
)

In [ ]:
alpha_metadata['alphashape_survey_date_1'].unique()

In [ ]:
unique_dates = sorted(alpha_metadata['alphashape_survey_date_1'].unique())
unique_dates

In [ ]:
def plot_alpha_shape_with_points_3transects_all_dates(transect_idxs, alpha_metadata, alpha_shapes, all_points):
    
    fig, axes = plt.subplots(1, 3, figsize=(20 * centimeters, 5 * centimeters), gridspec_kw={'wspace': 0.02}, dpi=600)
    
    unique_dates = sorted(alpha_metadata['alphashape_survey_date_1'].unique())
    colors = plt.cm.twilight_shifted(np.linspace(0, 0.9, 11))
    date_color_map = dict(zip(unique_dates, colors))
    
    all_x_vals = []
    all_z_vals = []
    
    for transect_idx in transect_idxs:
        transect_metadata = alpha_metadata[alpha_metadata['transect_idx'] == transect_idx]
        
        for _, row in transect_metadata.iterrows():
            date1, date2 = row['alphashape_survey_date_1'], row['alphashape_survey_date_2']
            
            transects = get_subset(all_points, transect_idx, date1, date2, 'survey_date')
            boundaries = get_subset(all_points, transect_idx, date1, date2, 'boundary_survey_date_1', 'boundary_survey_date_2')
            random = get_subset(all_points, transect_idx, date1, date2, 'random_survey_date_1', 'random_survey_date_2')
            
            aggregated = pd.concat([transects, boundaries, random], ignore_index=True)
            if not aggregated.empty:
                all_x_vals.extend(aggregated['x_piecewise'])
                all_z_vals.extend(aggregated['z_piecewise'])
    
    x_padding = (max(all_x_vals) - min(all_x_vals)) * 0.05
    z_padding = (max(all_z_vals) - min(all_z_vals)) * 0.05
    xlim = (min(all_x_vals) - x_padding, max(all_x_vals) + x_padding)
    zlim = (min(all_z_vals) - z_padding, max(all_z_vals) + z_padding)
    
    for plot_idx, transect_idx in enumerate(transect_idxs):
        ax = axes[plot_idx]
        
        transect_metadata = alpha_metadata[alpha_metadata['transect_idx'] == transect_idx]
        
        for _, row in transect_metadata.iterrows():
            date1, date2 = row['alphashape_survey_date_1'], row['alphashape_survey_date_2']
            poly = alpha_shapes[row['polygon_index']]
            survey_date_1 = row['alphashape_survey_date_1']
            
            color = date_color_map[survey_date_1]
            
            if isinstance(poly, Polygon):
                x, y = poly.exterior.xy
                ax.fill(x, y, alpha=0.3, facecolor=color, edgecolor=color, linewidth=1)
            elif isinstance(poly, MultiPolygon):
                for p in poly.geoms:
                    x, y = p.exterior.xy
                    ax.fill(x, y, alpha=0.3, facecolor=color, edgecolor=color, linewidth=1)
            
            transects = get_subset(all_points, transect_idx, date1, date2, 'survey_date')
            boundaries = get_subset(all_points, transect_idx, date1, date2, 'boundary_survey_date_1', 'boundary_survey_date_2')
            random = get_subset(all_points, transect_idx, date1, date2, 'random_survey_date_1', 'random_survey_date_2')

            # color *= 0.7
            
            ax.plot(transects['x_piecewise'], transects['z_piecewise'], '.', 
                markersize=1, alpha=1, color=color, rasterized=True, mew=0, zorder=10)
        
            ax.plot(boundaries['x_piecewise'], boundaries['z_piecewise'], '.', 
                markersize=1, alpha=0.7, color=color, rasterized=True, mew=0)
        
            ax.plot(random['x_piecewise'], random['z_piecewise'], '.', 
                markersize=1, alpha=0.7, color=color, rasterized=True, mew=0)
        
        ax.set_xlabel('distance along transect (m)' if plot_idx == 1 else '')  # Only middle subplot
        ax.set_ylabel('z (m)' if plot_idx == 0 else '')  # Only first subplot
        # ax.set_title(f'transect {transect_idx}')
        ax.set_xlim(xlim)
        ax.set_ylim(zlim)
        ax.tick_params(axis='both', pad=3)
        ax.set_xlim(-0.48, 13.7)
        ax.set_ylim(3.36, 11.55)
        
        if plot_idx > 0:
            ax.set_yticks([])
    
    # axes[0].legend(loc='upper left', markerscale=3, handletextpad=0.2)
    plt.tight_layout()
    os.makedirs('f2-alphashapes', exist_ok=True)
    transect_str = '_'.join(map(str, transect_idxs))
    fig.savefig(f'f2-alphashapes/alphashapes_all_dates_{transect_str}.pdf', 
               bbox_inches='tight', pad_inches=0.05, dpi=600)
    
    plt.show()

In [ ]:
plot_alpha_shape_with_points_3transects_all_dates(
    [21, 60, 96], 
    alpha_metadata, 
    alpha_shapes, 
    all_cleaned_perimeter_plus_random
)

In [ ]:
def plot_piecewise_sampled_vs_box_filtered_profile_3transects(df_box_filtered, df_piecewise_sampled, transect_idxs, dates):
    
    fig, axes = plt.subplots(1, 3, figsize=(20 * centimeters, 5 * centimeters), gridspec_kw={'wspace': 0.02})
    dates = pd.to_datetime(dates)
    colors = plt.cm.twilight_shifted(np.linspace(0, 0.9, len(dates)))
    
    all_x_vals = []
    all_z_vals = []
    
    for transect_idx in transect_idxs:
        for date in dates:
            f = df_box_filtered[
                (df_box_filtered['transect_idx'] == transect_idx) &
                (df_box_filtered['survey_date'] == date)
            ].dropna(subset=['x_box_median', 'z_box_median'])
            
            s = df_piecewise_sampled[
                (df_piecewise_sampled['transect_idx'] == transect_idx) & 
                (df_piecewise_sampled['survey_date'] == date)
            ]
            
            if not f.empty:
                all_x_vals.extend(f['x_box_median'])
                all_z_vals.extend(f['z_box_median'])
            if not s.empty:
                all_x_vals.extend(s['x_piecewise'])
                all_z_vals.extend(s['z_piecewise'])
    
    x_padding = (max(all_x_vals) - min(all_x_vals)) * 0.05
    z_padding = (max(all_z_vals) - min(all_z_vals)) * 0.05
    xlim = (min(all_x_vals) - x_padding, max(all_x_vals) + x_padding)
    zlim = (min(all_z_vals) - z_padding, max(all_z_vals) + z_padding)
    
    for plot_idx, transect_idx in enumerate(transect_idxs):
        ax = axes[plot_idx]
        
        for i, date in enumerate(dates):    
            f = df_box_filtered[
                (df_box_filtered['transect_idx'] == transect_idx) &
                (df_box_filtered['survey_date'] == date)
            ].dropna(subset=['x_box_median', 'z_box_median'])

            s = df_piecewise_sampled[
                (df_piecewise_sampled['transect_idx'] == transect_idx) & 
                (df_piecewise_sampled['survey_date'] == date)
            ]
            
            base_color = np.minimum(colors[i] + 0.1, 1.0)
            dark_color = base_color * 0.7

            if not f.empty:
                ax.plot(f['x_box_median'], f['z_box_median'], '.', alpha=0.1, markersize=6, color=base_color, mew=0, rasterized=True)
            if not s.empty:
                label = date.strftime('%Y-%m-%d') if plot_idx == 0 else None  # Only label first subplot
                ax.plot(s['x_piecewise'], s['z_piecewise'], '.', markersize=2, label=label, color=dark_color, mew=0, alpha=1, rasterized=True)

        # ax.set_xlabel('x (m)')
        ax.set_ylabel('z (m)' if plot_idx == 0 else '')  # Only label y-axis for first subplot
        ax.set_title(f'transect {transect_idx}')
        ax.set_xlim(xlim)
        ax.set_ylim(zlim)
        ax.tick_params(axis='both', pad=3)
        ax.set_xticks([])
        # ax.set_xticks([1, 4, 7, 9])
        # for ax in axes:
        #     ax.grid(axis='y', linestyle='-', alpha=0.5)

        if plot_idx > 0:
            ax.set_yticks([])
            # ax.spines['left'].set_visible(False)
            # ax.spines['right'].set_visible(False)
        if plot_idx != 1:
            ax.set_xlabel('')
    
    axes[0].legend(loc='upper left', bbox_to_anchor=(3.05, 1.05), markerscale=3, handletextpad=0.2)
    plt.tight_layout()
    
    os.makedirs('f2-alphashapes', exist_ok=True)
    transect_str = '_'.join(map(str, transect_idxs))
    fig.savefig(f'f2-alphashapes/transects_{transect_str}.pdf', bbox_inches='tight', pad_inches=0.05, dpi=600)
    
    plt.show()

plot_piecewise_sampled_vs_box_filtered_profile_3transects(
    all_transects_filt_add_max, 
    all_transects_sampled, 
    [21, 60, 96], 
    dates
)

In [ ]:
def plot_transect_evolution(transect_idx):
    transect_data = all_cleaned_perimeter_plus_random[all_cleaned_perimeter_plus_random['transect_idx'] == transect_idx]
    transect_dates = sorted(transect_data['survey_date'].dropna().unique())
    date_pairs = list(zip(transect_dates[:-1], transect_dates[1:]))

    fig, ax = plt.subplots(dpi=150)
    colors = plt.cm.tab10(np.linspace(0, 1, len(date_pairs)))
    
    plotted_pairs = []  # track successfully plotted pairs for color indexing

    for i, (date1, date2) in enumerate(date_pairs):
        date1, date2 = pd.to_datetime(date1), pd.to_datetime(date2)
        match = get_subset(alpha_metadata, transect_idx, date1, date2, 'alphashape_survey_date_1', 'alphashape_survey_date_2')
        
        # check if match exists (not empty after dropping failed rows)
        if match.empty:
            print(f"Skipping {transect_idx} {date1.date()} → {date2.date()}: no alphashape available")
            continue
            
        row = match.iloc[0]
        poly, area, alpha = alpha_shapes[row['polygon_index']], row['area'], row['alpha']
        color_idx = len(plotted_pairs)  # use number of plotted pairs for consistent coloring

        if isinstance(poly, Polygon):
            x, y = poly.exterior.xy
            ax.fill(x, y, alpha=0.3, facecolor=colors[color_idx], edgecolor='black', 
                   label=f'{date1.date()} → {date2.date()}, {area:.2f} m²')
        elif isinstance(poly, MultiPolygon):
            for j, p in enumerate(poly.geoms):
                x, y = p.exterior.xy
                label = f'{date1.date()} → {date2.date()}, {area:.2f} m²' if j == 0 else None
                ax.fill(x, y, alpha=0.3, facecolor=colors[color_idx], edgecolor='black', label=label)
                
        transects = get_subset(all_cleaned_perimeter_plus_random, transect_idx, date1, date2, 'survey_date')
        boundaries = get_subset(all_cleaned_perimeter_plus_random, transect_idx, date1, date2, 'boundary_survey_date_1', 'boundary_survey_date_2')
        random = get_subset(all_cleaned_perimeter_plus_random, transect_idx, date1, date2, 'random_survey_date_1', 'random_survey_date_2')

        ax.plot(transects['x_piecewise'], transects['z_piecewise'], '.', markersize=1, color=colors[color_idx], alpha=0.7)
        ax.plot(boundaries['x_piecewise'], boundaries['z_piecewise'], '.', markersize=1, color=colors[color_idx], alpha=0.7)
        ax.plot(random['x_piecewise'], random['z_piecewise'], '.', markersize=1, color=colors[color_idx], alpha=0.7)
        
        plotted_pairs.append((date1, date2))
    
    # only create plot if we have data to plot
    if plotted_pairs:
        ax.set_xlabel('x (m)')
        ax.set_ylabel('z (m)')
        ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0)
        ax.set_title(f'transect {transect_idx}')
        # ax.set_aspect('equal')

        os.makedirs('sandbox/check-alpha', exist_ok=True)
        fig.savefig(f'sandbox/check-alpha/transect_{transect_idx}', dpi=400, bbox_inches='tight')
    else:
        print(f"No valid alphashapes to plot for transect {transect_idx}")

    # plt.show()
    plt.close()

In [ ]:
# for t_idx in all_cleaned_perimeter_plus_random['transect_idx'].unique():
#    plot_transect_evolution(transect_idx=t_idx)

In [ ]:
alpha_metadata

In [ ]:
sorted_areas = np.sort(alpha_metadata['area'] / 6.5)
cumulative_freq = np.arange(1, len(sorted_areas) + 1)

# Plot
plt.plot(sorted_areas, cumulative_freq)
plt.xlabel('mean retreat distance (m)')
plt.ylabel('Cumulative Frequency')

In [ ]:
alpha_metadata['area'].describe()

In [ ]:
alpha_metadata['area'].describe()

In [ ]:
alpha_metadata.groupby('alphashape_survey_date_1')['area'].describe()

In [ ]:
alpha_metadata.groupby('transect_idx')['area'].min().describe()

In [ ]:
select_to_plot = alpha_metadata[
    alpha_metadata['alphashape_survey_date_1'] == pd.to_datetime('2022-04-15')
    ]

x

In [ ]:
unique_dates = alpha_metadata['alphashape_survey_date_1'].unique()
n_plots = len(unique_dates)

# create figure with shared axes
fig, axes = plt.subplots(n_plots, 1, figsize=(19 * centimeters, 1.5*n_plots * centimeters), 
                        sharex=True, sharey=True, gridspec_kw={'hspace':0.1})

# ensure axes is always a list (handles case of single subplot)
if n_plots == 1:
    axes = [axes]

for i, date in enumerate(unique_dates):
    date_data = alpha_metadata[alpha_metadata['alphashape_survey_date_1'] == date]
    date2 = date_data['alphashape_survey_date_2'].iloc[0]
    
    # plot data on current axis
    axes[i].scatter(date_data['transect_idx'], date_data['area'], marker='.', s=10, lw=0)
    
    # set title for each subplot
    # axes[i].set_title()
    
    # set axis limits (will be shared across all subplots)
    axes[i].set_ylim(0.1, 60)
    axes[i].set_xlim(-3, 131)
    
    center_idx = n_plots // 2
    if i == center_idx:
        axes[i].set_ylabel('cross-sectional area between outer channel bank profiles (m²)', labelpad=10)
    if i == n_plots - 1:  # last subplot
        axes[i].set_xlabel('transect #')
    
    if i != n_plots - 1:  # not the last subplot
        axes[i].tick_params(labelbottom=False)
    # if i != center_idx:  # not the center subplot
    #     axes[i].tick_params(labelleft=False)

    axes[i].annotate(f'{pd.Timestamp(date).date()} to {pd.Timestamp(date2).date()}', xy=(0.12, 0.9), xycoords='axes fraction', 
        ha='center', va='top', fontsize=7)

plt.tight_layout()
plt.show()
fig.savefig('transects.pdf', bbox_inches='tight', pad_inches=0.05)

In [ ]:
unique_dates = alpha_metadata['alphashape_survey_date_1'].unique()
n_plots = len(unique_dates)

fig, axes = plt.subplots(6, 2, figsize=(21 * centimeters, 5*2 * centimeters), 
                        sharex=True, sharey=True, 
                        gridspec_kw={'hspace':0.1, 'wspace':0.1})

for i, date in enumerate(unique_dates):
    row = i // 2  # integer division: 0,1→0, 2,3→1, 4,5→2, etc.
    col = i % 2   # modulo: 0,2,4,6,8→0, 1,3,5,7,9→1
    
    date_data = alpha_metadata[alpha_metadata['alphashape_survey_date_1'] == date]
    date2 = date_data['alphashape_survey_date_2'].iloc[0]
    
    # plot data using 2D indexing
    axes[row, col].scatter(date_data['transect_idx'], date_data['area'], 
                          marker='.', s=10, lw=0)
    
    # set axis limits
    axes[row, col].set_ylim(0.1, 60)
    axes[row, col].set_xlim(-3, 131)
    
    # ylabel only on left column
    # if col == 0:
    #     axes[row, col].set_ylabel('cross-sectional area between outer channel bank profiles (m²)', 
    #                              labelpad=10)
    
    # xlabel only on bottom row
    if row == 4:
        axes[row, col].set_xlabel('transect #')
    
    # hide tick labels except on edges
    if row != 4:
        axes[row, col].tick_params(labelbottom=False)
    # if col == 0:
    #     axes[row, col].tick_params(labelleft=False)
    # if col == 1:
    #     axes[row, col].tick_params(labelleft=False)

    axes[row, col].annotate(f'{pd.Timestamp(date).date()} to {pd.Timestamp(date2).date()}', 
                           xy=(0.2, 0.9), xycoords='axes fraction', 
                           ha='center', va='top', fontsize=7)

for i in range(n_plots, 10):
    row = i // 2
    col = i % 2
    axes[row, col].set_visible(False)

plt.tight_layout()
plt.show()
fig.savefig('transects.pdf', bbox_inches='tight', pad_inches=0.05)

#### examples

In [ ]:
alpha_val = 2.5
alpha_shape = alphashape.alphashape(points_2d, alpha_val)

fig, ax = plt.subplots(figsize=(7, 5), dpi=150)
ax.scatter(*zip(*points_2d), s=1, marker='.', label='sampled points')

x, y = alpha_shape.exterior.xy
ax.fill(x, y, alpha=0.2, color='gray', label=f'alpha = {alpha_val}')

ax.set(xlabel='x', ylabel='z', title='alpha shape overlay')
ax.legend()
plt.show()

In [ ]:
df_all = pd.concat([center_points, boundary_points], ignore_index=True)

points_2d = list(zip(df_all['x_piecewise'], df_all['z_piecewise']))

fig, ax = plt.subplots()
ax.scatter(*zip(*points_2d), s=2)
ax.set(xlabel='x', ylabel='z', title='points used for alpha shape')
plt.show()

In [ ]:
alpha_val = 5
alpha_shape = alphashape.alphashape(points_2d, alpha_val)

fig, ax = plt.subplots(figsize=(7, 5), dpi=150)
ax.scatter(*zip(*points_2d), s=1, marker='.', label='sampled points')

x, y = alpha_shape.exterior.xy
ax.fill(x, y, alpha=0.2, color='gray', label=f'alpha = {alpha_val}')

ax.set(xlabel='x', ylabel='z', title='alpha shape overlay')
ax.legend()
plt.show()

In [ ]:
area = alpha_shape.area
print(f'Area: {area:.2f} square units')

In [ ]:
# remove all z values less than the highest z_min for each transect (combat effect of stage height variation)

z_min_max = (
    all_transects
    .groupby(['transect_idx', 'survey_date'])['z'].min()
    .groupby('transect_idx').max()
    .rename('z_min_max')
    .reset_index()
)
all_transects = all_transects.merge(z_min_max, on='transect_idx')
all_transects = all_transects[all_transects['z'] >= all_transects['z_min_max']].copy()
all_transects = all_transects.drop(columns='z_min_max')

# adjust d_centerline to be relative to the min value for each transect



---

plot adjusted data

In [ ]:
def plot_transect_elevation(df, transect_idx=None, survey_date=None, bend=None):
    """
    Plot elevation profiles for a single transect (color by survey_date)
    or a single survey_date (color by transect_idx).
    Optionally filter by bend when survey_date is specified.
    """
    if (transect_idx is not None) and (survey_date is not None):
        raise ValueError("Specify only one of transect_idx or survey_date.")
    if (transect_idx is None) and (survey_date is None):
        raise ValueError("Specify either transect_idx or survey_date.")

    if transect_idx is not None:
        subset = df[df['transect_idx'] == transect_idx]
        color_col = 'survey_date'
        cmap = 'viridis'
        label = 'survey'
        title = f'transect {transect_idx}'
    else:
        subset = df[df['survey_date'] == pd.to_datetime(survey_date)]
        if bend is not None:
            subset = subset[subset['bend'] == bend]
        color_col = 'transect_idx'
        cmap = 'magma'
        label = 'transect number'
        title = f'survey on {survey_date}' + (f', bend {bend}' if bend is not None else '')

    if color_col == 'survey_date':
        color_vals = mdates.date2num(subset[color_col])
    else:
        color_vals = subset[color_col]

    fig, ax = plt.subplots(figsize=(15 * centimeters, 10 * centimeters))

    sc = ax.scatter(
        x=subset['d_centerline'],
        y=subset['z'],
        c=color_vals,
        cmap=cmap,
        alpha=0.5,
        marker='o',
        s=5,
    )
    cbar = plt.colorbar(sc, ax=ax, orientation='vertical', pad=0.03, aspect=15, shrink=0.5)
    cbar.set_label(label)

    if color_col == 'survey_date':
        cbar.ax.yaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

    ax.set_xlabel('distance along transect (m)')
    ax.set_ylabel('elevation (m)')
    ax.set_title(title, fontsize='medium')
    # ax.set_aspect('equal')
    plt.show()

In [ ]:
plot_transect_elevation(all_transects, transect_idx=58)

In [ ]:
plot_transect_elevation(all_transects, survey_date='2022-06-17', bend=1)

detour to investigate distribution of erosion values calculated with mean, median, IQR of d_centerline_adj per transect. suspicious from initial plots that point density effects are skewing results...

In [ ]:
# inves

group_cols = ['transect_idx', 'survey_date']

group_stats = (
    all_transects
    .groupby(['transect_idx', 'survey_date'])['d_centerline_adj']
    .agg(
        median='median',
        mean='mean',
        q25=lambda x: x.quantile(0.25),
        q75=lambda x: x.quantile(0.75)
    )
    .reset_index()
)

df_with_stats = all_transects.merge(group_stats, on=['transect_idx', 'survey_date'])

In [ ]:
transect_ids = df_with_stats['transect_idx'].unique()

# loop over transects
for transect_id in transect_ids[::10]:
    subset = df_with_stats[df_with_stats['transect_idx'] == transect_id]

    fig, ax = plt.subplots(figsize=(8, 4))

    ax.scatter(subset['survey_date'], subset['d_centerline_adj'], label='original points', s=8, alpha=0.5)
    ax.plot(subset['survey_date'], subset['median'], label='median', color='black', linewidth=1)
    ax.plot(subset['survey_date'], subset['mean'], label='mean', color='red', linestyle='--', linewidth=1)

    ax.fill_between(
        subset['survey_date'],
        subset['q25'],
        subset['q75'],
        color='gray',
        alpha=0.2,
        label='IQR (25–75p)'
    )

    ax.set_ylabel('d_centerline_adj (m)')
    ax.set_xlabel('survey date')
    ax.set_title(f'transect {transect_id}')
    ax.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()